# Selección y División de Datos con Scikit-Learn

Fundamental de seguir en la fase de preparación de los datos.

In [14]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, SelectFromModel, chi2
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# ALTERNATIVA: Dividir y cargar directamente los datos en X e y
X, y = load_iris(return_X_y=True)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [5]:
print(f'Tamaño del conjunto total: {len(X)}')
print(f'Tamaño del conjunto entrenamiento: {len(X_train)}')
print(f'Tamaño del conjunto pruebas: {len(X_test)}')

Tamaño del conjunto total: 150
Tamaño del conjunto entrenamiento: 120
Tamaño del conjunto pruebas: 30


In [6]:
# Obtener más información sobre train_test_split
help(train_test_split)

Help on function train_test_split in module sklearn.model_selection._split:

train_test_split(*arrays, test_size=None, train_size=None, random_state=None, shuffle=True, stratify=None)
    Split arrays or matrices into random train and test subsets.

    Quick utility that wraps input validation,
    ``next(ShuffleSplit().split(X, y))``, and application to input data
    into a single call for splitting (and optionally subsampling) data into a
    one-liner.

    Read more in the :ref:`User Guide <cross_validation>`.

    Parameters
    ----------
    *arrays : sequence of indexables with same length / shape[0]
        Allowed inputs are lists, numpy arrays, scipy-sparse
        matrices or pandas dataframes.

    test_size : float or int, default=None
        If float, should be between 0.0 and 1.0 and represent the proportion
        of the dataset to include in the test split. If int, represents the
        absolute number of test samples. If None, the value is set to the
        com

## Uso de `SelectKBest` y `SelectFromModel`

A veces, no se trata de separar solamente en grupos de prueba y de entrenamiento, sino de **identificar cuáles son las características más importantes** para hacer predicciones.

`SelectKBest` selecciona una cantidad determinada de características relevantes, basadas en pruebas estadísticas _univariadas_. Es muy útil cuando queremos mantener _solamente_ las características más importantes, mirándolas a cada una de ellas por separado.

In [8]:
selector: SelectKBest = SelectKBest(chi2, k=2)

Los parámetros más importantes son:

- `score_func`: la función de puntaje que se utilizará (e.g. [χ²](https://es.wikipedia.org/wiki/Prueba_%CF%87%C2%B2)).

- `k`: la cantidad de características que necesitamos.

In [12]:
X_train_selected = selector.fit_transform(X_train, y_train)
print(X_train[:5])
print(X_train_selected[:5])


[[4.6 3.6 1.  0.2]
 [5.7 4.4 1.5 0.4]
 [6.7 3.1 4.4 1.4]
 [4.8 3.4 1.6 0.2]
 [4.4 3.2 1.3 0.2]]
[[1.  0.2]
 [1.5 0.4]
 [4.4 1.4]
 [1.6 0.2]
 [1.3 0.2]]


In [13]:
X_test_selected = selector.transform(X_test)
print(X_test[:5])
print(X_test_selected[:5])

[[6.1 2.8 4.7 1.2]
 [5.7 3.8 1.7 0.3]
 [7.7 2.6 6.9 2.3]
 [6.  2.9 4.5 1.5]
 [6.8 2.8 4.8 1.4]]
[[4.7 1.2]
 [1.7 0.3]
 [6.9 2.3]
 [4.5 1.5]
 [4.8 1.4]]


Como se ve, han sido seleccionadas solamente dos columnas con las características más importantes, según la función de puntaje indicada.

`SelectFromModel` selecciona características relevantes basándose en la importancia de las características asignadas por un modelo predictivo base.

A diferencia de `SelectKBest` (que evalúa cada característica por sí misma, de manera independiente del resto), `SelectFromModel` sí considera la interacción que hay entre dichas características. Además, si no se especifica la cantidad de características que queremos en `SelectFromModel` (con el parámetro `threshold`), nos va a devolver tantas como la media de las importancias de las características calculadas por la media base. En general ,es preferible que `SelectFromModel` seleccione automáticamente cuántas características va a seleccionar.

In [15]:
# Modelo predictivo base
model: RandomForestClassifier = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

In [16]:
# Nuevo selector para el caso
selector_model = SelectFromModel(model)

In [17]:
X_train_selected_from_model = selector_model.fit_transform(X_train, y_train)

In [18]:
print(X_train[:5])
print(X_train_selected_from_model[:5])

[[4.6 3.6 1.  0.2]
 [5.7 4.4 1.5 0.4]
 [6.7 3.1 4.4 1.4]
 [4.8 3.4 1.6 0.2]
 [4.4 3.2 1.3 0.2]]
[[1.  0.2]
 [1.5 0.4]
 [4.4 1.4]
 [1.6 0.2]
 [1.3 0.2]]
